# Неделя 3. Соревнование.
В этом задании вам нужно воспользоваться опытом предыдущих недель, чтобы побить бейзлайн в [соревновании по сентимент-анализу отзывов](https://www.kaggle.com/c/product-reviews-sentiment-analysis-light) на товары на Kaggle Inclass.

In [45]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import random

In [4]:
# seed = random.randint(0, 10000)
seed = 8269
seed

8269

In [24]:
train_data = pd.read_csv(
    "..\..\Data\products_sentiment_train.tsv",
    "\t",
    names=["text", "class"],
    dtype={ "text": "str", "class": "int" })
train_data.head()

,text,class
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [25]:
test_data = pd.read_csv(
    "..\..\Data\products_sentiment_test.tsv",
    "\t",
    index_col="Id",
    dtype={ "text": "str", "Id": "int" })
test_data.head()

,text
Id,
0,"so , why the small digital elph , rather than ..."
1,3/4 way through the first disk we played on it...
2,better for the zen micro is outlook compatibil...
3,6 . play gameboy color games on it with goboy .
4,"likewise , i 've heard norton 2004 professiona..."


In [31]:
reviews_count = train_data.shape[0]
positive_reviews_proportion = float(train_data[train_data["class"] == 1].shape[0])/float(reviews_count)
print("Обучающая выборка:")
print("\tКоличество отзывов: {0}".format(reviews_count))
print("\tДоля позитивных отзывов: {0}".format(positive_reviews_proportion))

Обучающая выборка:
	Количество отзывов: 2000
	Доля позитивных отзывов: 0.637


In [42]:
texts = train_data["text"].tolist()
classes = train_data["class"].tolist()

# Бейзлайн модель

In [52]:
logistic_pipe = Pipeline([("vectorize", CountVectorizer()), ("model", LogisticRegression())])
accuracy_scores = cross_val_score(logistic_pipe, texts, classes, scoring="accuracy")
roc_auc_scores = cross_val_score(logistic_pipe, texts, classes, scoring="roc_auc")
print ("Accuracy:\n\tAverage: {0:.3f}\n\tStandard Deviation: {1:.3f}".format(
    np.average(accuracy_scores),
    np.std(accuracy_scores)))
print ("ROC AUC:\n\tAverage: {0:.3f}\n\tStandard Deviation: {1:.3f}".format(
    np.average(roc_auc_scores),
    np.std(roc_auc_scores)))

Accuracy:
	Average: 0.774
	Standard Deviation: 0.023
ROC AUC:
	Average: 0.832
	Standard Deviation: 0.020


In [55]:
vectorizer = CountVectorizer()
vectorized_texts = vectorizer.fit_transform(texts)
base_model = LogisticRegression().fit(vectorized_texts, classes)

main_features = list(map(
    lambda feature: feature[0],
    sorted(
        zip(vectorizer.get_feature_names(), np.abs(base_model.coef_[0])),
        key=lambda feature: feature[1],
        reverse=True)[:5]))
print ("Main features:", main_features)

Main features: ['great', 'love', 'excellent', 'easy', 'only']
